In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

import soundfile as sf

from pathlib import Path

In [ ]:
import scipy
import pandas as pd
import librosa
import warnings
import os

In [ ]:
import sys

# append the path of the
# parent directory
sys.path.append('..')
sys.path.append('../src/')
sys.path.append('../src/models/bat_call_detector/batdetect2/')

import src.batdt2_pipeline as batdetect2_pipeline
import src.file_dealer as file_dealer
from bat_detect.detector import models
from cfg import get_config
from pipeline import pipeline
from utils.utils import gen_empty_df, convert_df_ravenpro
from bat_detect.utils import wavfile
import bat_detect.utils.detector_utils as du
import bat_detect.detector.compute_features as feats
import bat_detect.detector.post_process as pp

In [ ]:
def plot_audio_seg(audio_features, spec_features):
    audio_seg = audio_features['audio_seg']
    fs = audio_features['sample_rate']
    start = audio_features['start']
    duration = audio_features['duration']

    vmax = spec_features['vmax']
    vmin = spec_features['vmin']
    cmap = spec_features['cmap']
    nfft = spec_features['NFFT']

    plt.figure(figsize=(15, 5))
    plt.rcParams.update({'font.size': 24})
    plt.title(spec_features['plot_title'], fontsize=24)
    # plt.specgram(audio_seg, NFFT=nfft, cmap=cmap)
    plt.specgram(audio_seg, NFFT=nfft, cmap=cmap, vmin=vmin, vmax=vmax)
    plt.yticks(ticks=np.linspace(0, 1, 6), labels=np.linspace(0, fs/2000, 6).astype('int'))
    plot_xtype = 'float'
    if (duration > 60):
        plot_xtype = 'int'
    plt.xticks(ticks=np.linspace(0, duration*(fs/2), 11), 
               labels=np.round(np.linspace(start, start+duration, 11, dtype=plot_xtype), 2), rotation=30)
    plt.ylabel("Frequency (kHz)")
    plt.xlabel("Time (s)")
    plt.show()

FREQ_COLORS = {'LF':'cyan',
               'HF':'orange'}

def plot_colored_dets_over_audio(audio_features, spec_features, plot_dets):
    audio_seg = audio_features['audio_seg']
    fs = audio_features['sample_rate']
    start = audio_features['start']
    duration = audio_features['duration']

    vmax = spec_features['vmax']
    vmin = spec_features['vmin']
    cmap = spec_features['cmap']
    nfft = spec_features['NFFT']

    plt.figure(figsize=(15, 5))
    plt.rcParams.update({'font.size': 24})
    plt.title(spec_features['plot_title'], fontsize=24)
    plt.specgram(audio_seg, NFFT=nfft, cmap=cmap, vmin=vmin, vmax=vmax)

    ax = plt.gca()
    for i, row in plot_dets.iterrows():
        rect = patches.Rectangle(((row['start_time'] - start)*(fs/2), row['low_freq']/(fs/2)), 
                        (row['end_time'] - row['start_time'])*(fs/2), (row['high_freq'] - row['low_freq'])/(fs/2), 
                        linewidth=2, edgecolor=FREQ_COLORS[row['KMEANS_CLASSES']], facecolor='none', alpha=0.8)
        ax.add_patch(rect)

    plt.yticks(ticks=np.linspace(0, 1, 6), labels=np.linspace(0, fs/2000, 6).astype('int'))
    plot_xtype = 'float'
    if (duration > 60):
        plot_xtype = 'int'
    plt.xticks(ticks=np.linspace(0, duration*(fs/2), 11), 
               labels=np.round(np.linspace(start, start+duration, 11, dtype=plot_xtype), 2), rotation=30)
    plt.ylabel("Frequency (kHz)")
    plt.xlabel("Time (s)")
    plt.show()

In [ ]:
files_from_loc = sorted(list(Path('/Users/adityakrishna/Downloads/array_test_20240830/').glob(pattern='20240830_20*.WAV')))
files_from_loc

In [ ]:
def run_pipeline_on_file(file, cfg):
    bd_preds = pd.DataFrame()

    if not cfg['output_dir'].is_dir():
        cfg['output_dir'].mkdir(parents=True, exist_ok=True)
    if not cfg['tmp_dir'].is_dir():
        cfg['tmp_dir'].mkdir(parents=True, exist_ok=True)


    if (cfg['run_model']):
        cfg["csv_filename"] = f"batdetect2_pipeline_{file.name.split('.')[0]}"
        filepath = (cfg['output_dir'] / f'{cfg["csv_filename"]}.csv')
        if not(filepath.is_file()):
            print(f"Generating detections for {file.name}")
            segmented_file_paths = batdetect2_pipeline.generate_segmented_paths([file], cfg)
            file_path_mappings = batdetect2_pipeline.initialize_mappings(segmented_file_paths, cfg)
            bd_preds = batdetect2_pipeline.run_models(file_path_mappings)
            if cfg['save']:
                batdetect2_pipeline._save_predictions(bd_preds, cfg['output_dir'], cfg)
            batdetect2_pipeline.delete_segments(segmented_file_paths)
        else:
            bd_preds = pd.read_csv(filepath)

    return bd_preds

In [ ]:
cfg = get_config()
cfg['tmp_dir'] = Path('../output/tmp')
cfg['output_dir'] = Path('../output_dir/mic_array_test_20240830/')
cfg['run_model'] = True
cfg['should_csv'] = True
cfg['save'] = True

In [ ]:
for file in files_from_loc:
    cfg['input_audio'] = Path(file)
    dets = run_pipeline_on_file(file, cfg)
    recorded_audio = sf.SoundFile(file)
    fs = recorded_audio.samplerate
    for start in np.arange(0, 50, 10):
        duration = 10
        recorded_audio.seek(int(fs*start))
        audio_seg = recorded_audio.read(int(fs*duration))
        lower_dB = -60
        upper_dB = 0
        vmin = lower_dB # hide anything below lower_dB dB
        vmax = upper_dB # hide anything above upper_dB dB
        audio_features = dict()
        audio_features['file_path'] = Path(file)
        audio_features['audio_seg'] = audio_seg
        audio_features['sample_rate'] = fs
        audio_features['start'] = start
        audio_features['duration'] = duration

        spec_features = dict()
        spec_features['NFFT'] = 1024 # When segments are short, NFFT should also be small to best see calls (must always be > 128)
        spec_features['cmap'] = 'jet' # This colormap shows best contrast between noise and signals
        spec_features['vmin'] = vmin
        spec_features['vmax'] = vmax

        spec_features['plot_title'] = f"Spectrogram of file: {audio_features['file_path'].name}"
        plot_dets = dets.loc[(dets['start_time']>=start)&(dets['end_time']<=(start+duration))]
        plot_colored_dets_over_audio(audio_features, spec_features, plot_dets)
        # plot_audio_seg(audio_features, spec_features)

In [ ]:
files_from_loc = sorted(list(Path('/Users/adityakrishna/Downloads/array_test_20240830/').glob(pattern='20240830_21*.WAV')))
files_from_loc

In [ ]:
for file in files_from_loc:
    cfg['input_audio'] = Path(file)
    dets = run_pipeline_on_file(file, cfg)
    recorded_audio = sf.SoundFile(file)
    fs = recorded_audio.samplerate
    for start in np.arange(0, (recorded_audio.frames/fs)-1, 10):
        recorded_audio.seek(int(fs*start))
        duration = min(10, (recorded_audio.frames - recorded_audio.tell()) / fs)
        audio_seg = recorded_audio.read(int(fs*duration))
        lower_dB = -60
        upper_dB = 0
        vmin = lower_dB # hide anything below lower_dB dB
        vmax = upper_dB # hide anything above upper_dB dB
        audio_features = dict()
        audio_features['file_path'] = Path(file)
        audio_features['audio_seg'] = audio_seg
        audio_features['sample_rate'] = fs
        audio_features['start'] = start
        audio_features['duration'] = duration

        spec_features = dict()
        spec_features['NFFT'] = 1024 # When segments are short, NFFT should also be small to best see calls (must always be > 128)
        spec_features['cmap'] = 'jet' # This colormap shows best contrast between noise and signals
        spec_features['vmin'] = vmin
        spec_features['vmax'] = vmax

        spec_features['plot_title'] = f"Spectrogram of file: {audio_features['file_path'].name}"
        plot_dets = dets.loc[(dets['start_time']>=start)&(dets['end_time']<=(start+duration))]
        plot_colored_dets_over_audio(audio_features, spec_features, plot_dets)
        # plot_audio_seg(audio_features, spec_features)

In [ ]:
files_from_loc = sorted(list(Path('/Users/adityakrishna/Downloads/array_test_20240830/').glob(pattern='20240830_22*.WAV')))
files_from_loc

In [ ]:
for file in files_from_loc:
    cfg['input_audio'] = Path(file)
    dets = run_pipeline_on_file(file, cfg)
    recorded_audio = sf.SoundFile(file)
    fs = recorded_audio.samplerate
    for start in np.arange(0, (recorded_audio.frames/fs)-1, 10):
        recorded_audio.seek(int(fs*start))
        duration = min(10, (recorded_audio.frames - recorded_audio.tell()) / fs)
        audio_seg = recorded_audio.read(int(fs*duration))
        lower_dB = -60
        upper_dB = 0
        vmin = lower_dB # hide anything below lower_dB dB
        vmax = upper_dB # hide anything above upper_dB dB
        audio_features = dict()
        audio_features['file_path'] = Path(file)
        audio_features['audio_seg'] = audio_seg
        audio_features['sample_rate'] = fs
        audio_features['start'] = start
        audio_features['duration'] = duration

        spec_features = dict()
        spec_features['NFFT'] = 1024 # When segments are short, NFFT should also be small to best see calls (must always be > 128)
        spec_features['cmap'] = 'jet' # This colormap shows best contrast between noise and signals
        spec_features['vmin'] = vmin
        spec_features['vmax'] = vmax

        spec_features['plot_title'] = f"Spectrogram of file: {audio_features['file_path'].name}"
        plot_dets = dets.loc[(dets['start_time']>=start)&(dets['end_time']<=(start+duration))]
        plot_colored_dets_over_audio(audio_features, spec_features, plot_dets)
        # plot_audio_seg(audio_features, spec_features)